# The Price is Right

Today we'll build another piece of the puzzle: a ScanningAgent that looks for promising deals by subscribing to RSS feeds.

In [2]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [3]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [4]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 5/5 [02:02<00:00, 24.49s/it]


In [5]:
len(deals)

50

In [8]:
deals[44].describe()

"Title: Garwinner 20'' 2-Stroke Cordless Gas Chainsaw for $104 + free shipping\nDetails: Cut wood at the farm, garden, or ranch. It's $86 off and the best deal we could find. Buy Now at Walmart\nFeatures: non-slip handle 2-stroke motor safety switch Model: 20IN-6220G\nURL: https://www.dealnews.com/products/Garwinner-20-2-Stroke-Cordless-Gas-Chainsaw/487987.html?iref=rss-c196"

In [9]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [10]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [11]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Soundcore Spring Deals: Up to 60% off + free shipping
Details: Soundcore members get early access to their Spring sale, which includes an extra 5% off select items, free gifts with certain orders, freebies with Soundcore credit redemptions, and more. (Not a member? It's free to join.) Plus, the 5% off discounts stack with the coupons listed alongside many of those h

In [12]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [13]:
result = get_recommendations()

In [14]:
len(result.deals)

5

In [17]:
result.deals[2]

Deal(product_description='The Samsung T5 EVO is a robust 4TB portable SSD that offers impressive storage and speed with USB 3.2 Gen 1 technology. It features shock resistance and fall protection up to 6 feet, ensuring your data is safe on the go. Moreover, it comes with 256-bit AES hardware encryption and delivers read and write speeds of up to 460MBps, making it an excellent choice for data-heavy applications.', price=220.0, url='https://www.dealnews.com/products/Samsung/Samsung-T5-EVO-4-TB-Portable-SSD/480881.html?iref=rss-c39')

In [18]:
from agents.scanner_agent import ScannerAgent

In [19]:
agent = ScannerAgent()
result = agent.scan()

In [21]:
result

DealSelection(deals=[Deal(product_description='The Ulefone Armor 28 Ultra 5G is a rugged Android smartphone designed for durability and performance. It features a MediaTek Dimensity 9300+ processor, a vibrant 6.67" AMOLED display, and a hefty 32GB of RAM for seamless multitasking. With its massive 1TB UFS4.0 storage, you\'ll have plenty of space for apps, photos, and videos. Plus, its IP68 rating means it\'s waterproof and resistant to dust, making it perfect for outdoor enthusiasts or those in demanding work environments.', price=870.0, url='https://www.dealnews.com/Ulefone-Armor-28-Ultra-5-G-AI-Thermal-Imaging-Rugged-Android-Phone-for-870-free-shipping/21725240.html?iref=rss-c142'), Deal(product_description="Stay connected with the Certified Refurb Status Between 3 ANC Wireless Earbuds, perfect for music lovers who value quality and comfort. These earbuds feature advanced active noise cancellation, which ensures an immersive listening experience without distractions. With a long-last